<a href="https://colab.research.google.com/github/Akash-AI-17/Real-time-stock-prediction---signal-generation/blob/main/Copy_of_stock_recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
import numpy as np
import plotly.graph_objects as go
import yfinance as yf
from datetime import datetime
from fuzzywuzzy import fuzz
import gradio as gr
from datetime import date, timedelta


/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Artificial News Generation

In [ ]:


companies = ["Reliance Industries", "Tata Consultancy Services", "HDFC Bank", "Infosys", "ICICI Bank", "Hindustan Unilever", "Bajaj Finance", "Kotak Mahindra Bank", "HDFC", "Bharti Airtel", "Axis Bank", "Maruti Suzuki", "Asian Paints", "Nestle India", "UltraTech Cement", "IndusInd Bank", "Tech Mahindra", "ITC", "Sun Pharmaceutical", "Power Grid Corporation of India"]

positive_news = [
    "Shares of {} surged today after the company announced record-breaking quarterly profits.",
    "Investors cheered as {} reported strong growth in its revenue and market share.",
    "{} stock jumped significantly following positive market sentiment and strong quarterly earnings.",
    "Analysts upgraded the rating for {} after the company exceeded earnings expectations.",
    "Market optimism propelled {} to new highs as investors remain bullish on its growth prospects."
]

negative_news = [
    "Investor confidence wavered as {} reported a decline in quarterly earnings.",
    "Market volatility led to a sharp decline in {} stock prices despite the company's efforts to streamline operations.",
    "Concerns over economic slowdown weighed heavily on {} shares, leading to a significant drop in value.",
    "Investor sentiment soured after {} issued a profit warning, citing challenges in the current market environment.",
    "Rising competition and regulatory challenges contributed to a downturn in {} stock performance."
]

neutral_news = [
    "{} maintained steady performance in today's trading session amid mixed market conditions.",
    "Market analysts remain neutral on {} stock, awaiting further developments in the sector.",
    "Investor sentiment towards {} remained unchanged as the company continues to navigate market challenges.",
    "Shares of {} showed minimal movement today as investors adopted a wait-and-see approach.",
    "{} stock experienced moderate trading activity today, reflecting overall market stability."
]

news_articles = []

for _ in range(20):
    company = random.choice(companies)
    tone = random.choice(["positive", "negative", "neutral"])
    if tone == "positive":
        news = random.choice(positive_news).format(company)
    elif tone == "negative":
        news = random.choice(negative_news).format(company)
    else:
        news = random.choice(neutral_news).format(company)
    news_articles.append(news)

print(news_articles)


['Investors cheered as Kotak Mahindra Bank reported strong growth in its revenue and market share.', 'Market analysts remain neutral on UltraTech Cement stock, awaiting further developments in the sector.', 'Investor confidence wavered as Axis Bank reported a decline in quarterly earnings.', 'Shares of Asian Paints showed minimal movement today as investors adopted a wait-and-see approach.', 'Shares of Bajaj Finance surged today after the company announced record-breaking quarterly profits.', "UltraTech Cement maintained steady performance in today's trading session amid mixed market conditions.", 'Shares of HDFC Bank showed minimal movement today as investors adopted a wait-and-see approach.', 'Asian Paints stock experienced moderate trading activity today, reflecting overall market stability.', 'Investor sentiment soured after HDFC Bank issued a profit warning, citing challenges in the current market environment.', 'Investor confidence wavered as Asian Paints reported a decline in qu

In [ ]:
news_articles[0]

'Investors cheered as Kotak Mahindra Bank reported strong growth in its revenue and market share.'

In [ ]:
news = pd.DataFrame(news_articles, columns=['News'])
news

,News
0,Investors cheered as Kotak Mahindra Bank repor...
1,Market analysts remain neutral on UltraTech Ce...
2,Investor confidence wavered as Axis Bank repor...
3,Shares of Asian Paints showed minimal movement...
4,Shares of Bajaj Finance surged today after the...
5,UltraTech Cement maintained steady performance...
6,Shares of HDFC Bank showed minimal movement to...
7,Asian Paints stock experienced moderate tradin...
8,Investor sentiment soured after HDFC Bank issu...
9,Investor confidence wavered as Asian Paints re...


Entity Recognition

In [ ]:
def extract_entities(df):
    tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
    model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

    nlp = pipeline("ner", model=model, tokenizer=tokenizer)

    entities = []
    for news in df['News']:
        ner_results = nlp(news)
        entities_in_news = []
        current_entity = ""
        for entity in ner_results:
            if entity['entity'][:2] == 'B-':
                if current_entity:
                    entities_in_news.append(current_entity.replace(" ##", ""))
                    current_entity = ""
                current_entity = entity['word']
            elif entity['entity'][:2] == 'I-':
                if current_entity:
                    current_entity += " " + entity['word']
                else:
                    current_entity = entity['word']
            else:
                if current_entity:
                    entities_in_news.append(current_entity.replace(" ##", ""))
                    current_entity = ""
        if current_entity:
            entities_in_news.append(current_entity.replace(" ##", ""))

        entities.append(entities_in_news)

    # Add new columns for entities to the DataFrame
    for i in range(len(entities[0])):
        #df[f'Entity_{i+1}'] = [entity_list[i] if len(entity_list) > i else None for entity_list in entities]
        df['entity'] = [entity_list[i] if len(entity_list) > i else None for entity_list in entities]
    return df


In [ ]:
df1 = extract_entities(news)
df1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


,News,entity
0,Investors cheered as Kotak Mahindra Bank repor...,Kotak Mahindra Bank
1,Market analysts remain neutral on UltraTech Ce...,UltraTech Cement
2,Investor confidence wavered as Axis Bank repor...,Axis Bank
3,Shares of Asian Paints showed minimal movement...,Asian Paints
4,Shares of Bajaj Finance surged today after the...,Bajaj Finance
5,UltraTech Cement maintained steady performance...,UltraTech Cement
6,Shares of HDFC Bank showed minimal movement to...,HDFC Bank
7,Asian Paints stock experienced moderate tradin...,Asian Paints
8,Investor sentiment soured after HDFC Bank issu...,HDFC Bank
9,Investor confidence wavered as Asian Paints re...,Asian Paints


SENTIMENT SCORING

In [ ]:
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [ ]:
nlp = pipeline("text-classification", model=finbert, tokenizer=tokenizer)

In [ ]:
def add_sentiment_column(df):


    # Get the sentiment for each news headline in the DataFrame
    sentiments = nlp(df['News'].tolist())

    # Extract labels and scores from the sentiment predictions
    labels = [result['label'] for result in sentiments]
    scores = [result['score'] for result in sentiments]

    # Add new columns 'Sentiment' and 'Sentiment Score' to the DataFrame
    df['Sentiment'] = labels
    df['Sentiment Score'] = scores

    # Reorder columns for a clean structure (optional)
    df = df[['News','entity','Sentiment', 'Sentiment Score']]

    return df

In [ ]:
add_sentiment_column(df1)

,News,entity,Sentiment,Sentiment Score
0,Investors cheered as Kotak Mahindra Bank repor...,Kotak Mahindra Bank,Positive,1.000000
1,Market analysts remain neutral on UltraTech Ce...,UltraTech Cement,Negative,0.616381
2,Investor confidence wavered as Axis Bank repor...,Axis Bank,Negative,0.999998
3,Shares of Asian Paints showed minimal movement...,Asian Paints,Neutral,0.987883
4,Shares of Bajaj Finance surged today after the...,Bajaj Finance,Positive,1.000000
5,UltraTech Cement maintained steady performance...,UltraTech Cement,Positive,1.000000
6,Shares of HDFC Bank showed minimal movement to...,HDFC Bank,Neutral,0.988738
7,Asian Paints stock experienced moderate tradin...,Asian Paints,Positive,0.994756
8,Investor sentiment soured after HDFC Bank issu...,HDFC Bank,Negative,1.000000
9,Investor confidence wavered as Asian Paints re...,Asian Paints,Negative,0.999985


Positive: range (0,1) | Negative: range (0,1) | Neutral: range (0,1).


Note: Assign range(0,1) separately for each sentiment.

CSV for fuzzy matching

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Stock_names.csv')
df

,Company Name,NSE,NSE CODE
0,Reliance Industries Ltd.,RELIANCE.NS,RELIANCE
1,Tata Consultancy Services Ltd.,TCS.NS,TCS
2,HDFC Bank Ltd.,HDFCBANK.NS,HDFCBANK
3,Infosys Ltd.,INFY.NS,INFY
4,Hindustan Unilever Ltd.,HINDUNILVR.NS,HINDUNILVR
...,...,...,...
1236,Consolidated Finvest & Holdings Ltd.,CONSOFINVT.NS,CONSOFINVT
1237,Hindustan Media Ventures Ltd.,HMVL.NS,HMVL
1238,Finkurve Financial Services Ltd.,.NS,NaN
1239,Asian Energy Services Ltd.,ASIANENE.NS,ASIANENE


In [ ]:
# Columns to be dropped
columns_to_drop = ['NSE CODE']

# Drop specified columns
df = df.drop(columns=columns_to_drop)

In [ ]:
new_col = {'Company Name':'Companies','NSE':'Symbols'}
df.rename(columns = new_col, inplace = True)

In [ ]:
df

,Companies,Symbols
0,Reliance Industries Ltd.,RELIANCE.NS
1,Tata Consultancy Services Ltd.,TCS.NS
2,HDFC Bank Ltd.,HDFCBANK.NS
3,Infosys Ltd.,INFY.NS
4,Hindustan Unilever Ltd.,HINDUNILVR.NS
...,...,...
1236,Consolidated Finvest & Holdings Ltd.,CONSOFINVT.NS
1237,Hindustan Media Ventures Ltd.,HMVL.NS
1238,Finkurve Financial Services Ltd.,.NS
1239,Asian Energy Services Ltd.,ASIANENE.NS


Extraction of stock's OHLCV data which are present in the news from yfinance.

Now we will visualize the data in candlesticks pattern and generate buy and sell signals on the chart.

MAIN SWING STRATEGY

In [ ]:
# Function to fetch data for last n days from yfinance for a single symbol
def fetch_data_for_symbol(symbol, interval, n_days):
    print(f"Fetching data for: {symbol}, timeframe={interval}, last {n_days} days")

    # Set end date as today
    end_date = pd.Timestamp.today().strftime("%Y-%m-%d")

    start_date = (pd.Timestamp.today() - pd.DateOffset(days=n_days)).strftime("%Y-%m-%d")

    data = yf.download(symbol, start=start_date, end=end_date, interval=interval)
    return data

# Step 1: Calculate the EMA
def calculate_ema(prices, window):
    weights = np.exp(np.linspace(-1., 0., window))
    weights /= weights.sum()
    ema = np.convolve(prices, weights, mode='full')[:len(prices)]
    ema[:window] = ema[window]
    return ema

# Step 3: Generate Buy signals
def generate_buy_signals(ema_1st, ema_2nd):
    buy_signals = np.where((ema_1st.shift(1) <= ema_2nd.shift(1)) & (ema_1st > ema_2nd), 1, 0)
    return buy_signals

# Step 4: Generate Sell signals
def generate_sell_signals(ema_1st, ema_2nd):
    sell_signals = np.where((ema_1st.shift(1) >= ema_2nd.shift(1)) & (ema_1st < ema_2nd), 1, 0)
    return sell_signals

# Function for fuzzy matching
def fuzzy_matching(stock_name, stocks):
    best_match = None
    best_match_ratio = 0

    for key in stocks.keys():
        ratio = fuzz.token_set_ratio(stock_name.lower(), key.lower())
        if ratio > best_match_ratio:
            best_match_ratio = ratio
            best_match = key

    return stocks.get(best_match)

# User input for number of days to fetch data
n_days = int(input("Enter the number of days of data you want to extract: "))

# User input for EMA values
ema_1st_value = int(input("Enter the value for 1st EMA: "))
ema_2nd_value = int(input("Enter the value for 2nd EMA: "))

# Initialize an empty list to store matched companies
matched_companies = []

# Iterate over rows of df1
for index, row in df1.iterrows():
    entity_name = row['entity']

    # Use fuzzy matching to find the corresponding symbol
    matched_symbol = fuzzy_matching(entity_name, dict(zip(df['Companies'], df['Symbols'])))

    if matched_symbol is not None and entity_name not in matched_companies:
        # Add the symbol to the list of matched companies
        matched_companies.append(entity_name)

        # Fetch data for the symbol
        df_symbol = fetch_data_for_symbol(matched_symbol, interval='1d', n_days=n_days)

        # Calculate EMAs
        df_symbol[f'{ema_1st_value} EMA'] = calculate_ema(df_symbol['Close'], ema_1st_value)
        df_symbol[f'{ema_2nd_value} EMA'] = calculate_ema(df_symbol['Close'], ema_2nd_value)

        # Generate Buy and Sell signals
        df_symbol['Buy Signal'] = generate_buy_signals(df_symbol[f'{ema_1st_value} EMA'], df_symbol[f'{ema_2nd_value} EMA'])
        df_symbol['Sell Signal'] = generate_sell_signals(df_symbol[f'{ema_1st_value} EMA'], df_symbol[f'{ema_2nd_value} EMA'])

        # Create traces
        fig = go.Figure()

        # Candlestick trace
        fig.add_trace(go.Candlestick(x=df_symbol.index,
                                    open=df_symbol['Open'],
                                    high=df_symbol['High'],
                                    low=df_symbol['Low'],
                                    close=df_symbol['Close'],
                                    name='Candlestick'))

        # EMA traces
        fig.add_trace(go.Scatter(x=df_symbol.index, y=df_symbol[f'{ema_1st_value} EMA'], mode='lines', name=f'{ema_1st_value} EMA'))
        fig.add_trace(go.Scatter(x=df_symbol.index, y=df_symbol[f'{ema_2nd_value} EMA'], mode='lines', name=f'{ema_2nd_value} EMA'))

        # Buy and Sell signals
        fig.add_trace(go.Scatter(x=df_symbol.index[df_symbol['Buy Signal'] == 1], y=df_symbol['Close'][df_symbol['Buy Signal'] == 1],
                                mode='markers', marker=dict(color='blue', symbol='triangle-up', size=10),
                                name='Buy Signal'))
        fig.add_trace(go.Scatter(x=df_symbol.index[df_symbol['Sell Signal'] == 1], y=df_symbol['Close'][df_symbol['Sell Signal'] == 1],
                                mode='markers', marker=dict(color='black', symbol='triangle-down', size=10),
                                name='Sell Signal'))

        # Update layout
        fig.update_layout(title=f'EMA Trading Strategy for {entity_name}',
                          xaxis_title='Date',
                          yaxis_title='Price',
                          hovermode='x unified',
                          showlegend=True)

        # Show plot
        fig.show()


Enter the number of days of data you want to extract: 1000
Enter the value for 1st EMA: 50
Enter the value for 2nd EMA: 100


[*********************100%%**********************]  1 of 1 completed

Fetching data for: KOTAKBANK.NS, timeframe=1d, last 1000 days


[*********************100%%**********************]  1 of 1 completed

Fetching data for: ULTRACEMCO.NS, timeframe=1d, last 1000 days


...

Fetching data for: NESTLEIND.NS, timeframe=1d, last 1000 days


[*********************100%%**********************]  1 of 1 completed

Fetching data for: RELIANCE.NS, timeframe=1d, last 1000 days


[*********************100%%**********************]  1 of 1 completed

Fetching data for: POWERGRID.NS, timeframe=1d, last 1000 days


GRADIO INTERFACE WITH CANDLESTICKS PATTERNS

In [ ]:


# Assuming df1 and df are predefined DataFrames with company information

def fetch_data_for_symbol(symbol, interval, n_days):
    try:
        end_date = pd.Timestamp.today().strftime("%Y-%m-%d")
        start_date = (pd.Timestamp.today() - pd.DateOffset(days=n_days)).strftime("%Y-%m-%d")
        data = yf.download(symbol, start=start_date, end=end_date, interval=interval)
        if data.empty:
            return None, "Failed to fetch data. The data might be empty."
        return data, None
    except Exception as e:
        return None, str(e)

def calculate_ema(prices, window):
    weights = np.exp(np.linspace(-1., 0., window))
    weights /= weights.sum()
    ema = np.convolve(prices, weights, mode='full')[:len(prices)]
    ema[:window] = ema[window]
    return ema

def generate_buy_sell_signals(df_symbol, ema_1st_value, ema_2nd_value):
    df_symbol['Buy Signal'] = np.where((df_symbol[f'{ema_1st_value} EMA'].shift(1) <= df_symbol[f'{ema_2nd_value} EMA'].shift(1)) & (df_symbol[f'{ema_1st_value} EMA'] > df_symbol[f'{ema_2nd_value} EMA']), 1, 0)
    df_symbol['Sell Signal'] = np.where((df_symbol[f'{ema_1st_value} EMA'].shift(1) >= df_symbol[f'{ema_2nd_value} EMA'].shift(1)) & (df_symbol[f'{ema_1st_value} EMA'] < df_symbol[f'{ema_2nd_value} EMA']), 1, 0)
    return df_symbol

def create_figure(df_symbol, ema_1st_value, ema_2nd_value, entity_name):
    fig = go.Figure()

    # Use Candlestick chart instead of line chart
    fig.add_trace(go.Candlestick(x=df_symbol.index,
                                 open=df_symbol['Open'],
                                 high=df_symbol['High'],
                                 low=df_symbol['Low'],
                                 close=df_symbol['Close'],
                                 name='Candlestick'))

    fig.add_trace(go.Scatter(x=df_symbol.index, y=df_symbol[f'{ema_1st_value} EMA'], mode='lines', name=f'{ema_1st_value} EMA'))
    fig.add_trace(go.Scatter(x=df_symbol.index, y=df_symbol[f'{ema_2nd_value} EMA'], mode='lines', name=f'{ema_2nd_value} EMA'))
    fig.add_trace(go.Scatter(x=df_symbol.index[df_symbol['Buy Signal'] == 1], y=df_symbol['Close'][df_symbol['Buy Signal'] == 1], mode='markers', marker=dict(color='blue', symbol='triangle-up',size=10), name='Buy Signal'))
    fig.add_trace(go.Scatter(x=df_symbol.index[df_symbol['Sell Signal'] == 1], y=df_symbol['Close'][df_symbol['Sell Signal'] == 1], mode='markers', marker=dict(color='black', symbol='triangle-down',size=10), name='Sell Signal'))

    fig.update_layout(title=f'Candlestick and EMA Trading Strategy for {entity_name}', xaxis_title='Date', yaxis_title='Price', hovermode='x unified', showlegend=True)

    return fig

def main(entity_name, n_days, ema_1st_value, ema_2nd_value):
    matched_symbol = entity_name  # Implement fuzzy matching as needed.
    df_symbol, error = fetch_data_for_symbol(matched_symbol, interval='1d', n_days=n_days)
    if error:
        return create_error_figure(f"Error: {error}")

    if df_symbol is not None and not df_symbol.empty:
        try:
            df_symbol['Close'] = pd.to_numeric(df_symbol['Close'], errors='coerce')  # Ensure 'Close' is numeric for calculation
            df_symbol[f'{ema_1st_value} EMA'] = calculate_ema(df_symbol['Close'], ema_1st_value)
            df_symbol[f'{ema_2nd_value} EMA'] = calculate_ema(df_symbol['Close'], ema_2nd_value)
            df_symbol = generate_buy_sell_signals(df_symbol, ema_1st_value, ema_2nd_value)
            fig = create_figure(df_symbol, ema_1st_value, ema_2nd_value, entity_name)
            return fig
        except ValueError as e:
            return create_error_figure(f"Error: {str(e)}")
    else:
        return create_error_figure("Error: No data available for the given symbol.")

def create_error_figure(error_message):
    fig = go.Figure()
    fig.add_annotation(text=error_message, xref="paper", yref="paper", showarrow=False, font=dict(size=20, color="red"))
    fig.update_layout(xaxis_visible=False, yaxis_visible=False)
    return fig

iface = gr.Interface(fn=main,
                     inputs=[gr.Textbox(label="Entity Name"), gr.Number(label="Number of Days"), gr.Number(label="1st EMA Value"), gr.Number(label="2nd EMA Value")],
                     outputs=gr.Plot(label="Candlestick and EMA Strategy Plot"),  # Updated plot label
                     description="Enter the company's name, number of days for data, and two EMA values to plot the EMA trading strategy.")
iface.launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://3ca36384055bc2f038.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


[*********************100%%**********************]  1 of 1 completed


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://3ca36384055bc2f038.gradio.live
